In [1]:
#@title Connect to Drive
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive/')

# # Authenticate Google Drive API
from google.colab import auth
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread
auth.authenticate_user() # Authenticate user credentials
creds, _ = default() # Get credentials
gc = gspread.authorize(creds) # Authorize credentials
import gspread_dataframe as gd # Import module 


!pip install PyPDF2

import pandas as pd # Import pandas library 
import shutil # Import shutil module for file operations
import os # Import os module 
import datetime # Import datetime module fordate and time data
import glob # Import glob module 
import re # Import re module 

from PyPDF2 import PdfReader # Import PdfReader module 


Mounted at /content/drive/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.7 MB/s eta 0:00:00


# Zomato Invoices

In [ ]:
#@title if files not in folder then move files from one folder to another 
destination = "/content/drive/MyDrive/02_Aggregator_Invoices/01_Zomato_Invoices"
sorurce = "/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Zomato/Zomato Invoices"


# ******************************************************** MAIN CODE ***********************************************************

count = 0

# Loop through each file in the source directory
for file in os.listdir(sorurce):
    src_path = os.path.join(sorurce, file)
    dest_path = os.path.join(destination, file)
    if not os.path.exists(dest_path): 
#        shutil.move(src_path, dest_path)
        count += 1

# Print the total number of files moved
print(f"{count} files moved")

0 files moved


In [ ]:
#@title Renaming pdf files to standard name by reading text inside and renaming
path = "/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Zomato/Zomato Invoices"





# ************************************************************************************* MAIN CODE **************************************************************************
files = glob.glob(os.path.join(path, "*.pdf"))

data1 = []
count = 0

for file in files:
    with open(file, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            
            # Extract Restaurant ID
            restaurant_id_pattern = r'Restaurant ID\s*:\s*(\d+)'
            restaurant_id_match = re.search(restaurant_id_pattern, text)
            restaurant_id = restaurant_id_match.group(1)
            
            # Extract Period
            period_pattern = r'Period\s*:\s*(\d+-\w+-\d+)\s*To\s*(\d+-\w+-\d+)'
            period_match = re.search(period_pattern, text)
            start_date = period_match.group(1)
            end_date = period_match.group(2)
            
            data1.append({'Restaurant ID': restaurant_id,
                         'Start Date': start_date,
                         'End Date': end_date})
        for i in data1:
            oldname = os.path.join(path,file)        
            split_name = os.path.splitext(os.path.basename(file))[0][:15]
            new_name = split_name + "_"+i['Restaurant ID'] + "_" +i['Start Date'].split("-")[1]+i['Start Date'].split("-")[2]+ '.pdf'             
            newname = os.path.join(path, new_name)     
        os.rename(oldname, newname)

        count+=1
print(count)

4


In [ ]:
#@title Main Code DC Zomato Invoices -1 💎
path='/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Zomato/Zomato Invoices'  # paste ur link here





# *************************************************************************************** MAIN CODE **************************************************************************************************
files = glob.glob(os.path.join(path, "*.pdf"))

count = 0

zom_data=pd.DataFrame()

for file in files:
  with open(file, 'rb') as pdf_file:
      pdf_reader = PdfReader(pdf_file)
      num_pages = len(pdf_reader.pages)
      text = ""
      count+=1
      for page_num in range(num_pages):
          page = pdf_reader.pages[page_num]
          text = page.extract_text()


      # Define regular expressions to extract information

      invoice_no_re = r"Invoice No : (\S+)"
      invoice_date_re = r"Invoice Date : (\S+)"
      trade_name_re = r"Trade Name : (.+)"
      period_re = r"Period: (.+)"
      convenience_fee_re = r"1 Convenience Fee (\S+)"
      commission_re = r"2 Commission for usage of Zomato's Online Ordering Platform (\S+)"
      taxable_amount_re = r"Taxable Amount (\S+)"
      sgst_re = r"SGST @ 9.00% (\S+)"
      cgst_re = r"CGST @ 9.00% (\S+)"
      total_amount_re = r"Total Amount (\S+)"

      # Extract information using regular expressions
      invoice_no = re.search(invoice_no_re, text).group(1)
      invoice_date = re.search(invoice_date_re, text).group(1)
      trade_name = re.search(trade_name_re, text).group(1)
      period = re.search(period_re, text).group(1)
      convenience_fee = str(re.search(convenience_fee_re, text).group(1))
      commission = float(re.search(commission_re, text).group(1).replace(',', ''))
      taxable_amount = float(re.search(taxable_amount_re, text).group(1).replace(',', ''))
      sgst = re.search(sgst_re, text).group(1)
      cgst = re.search(cgst_re, text).group(1)
      total_amount = float(re.search(total_amount_re, text).group(1).replace(',', ''))

      # Create dictionary of extracted information
      data = {'Invoice No': [invoice_no],
              'Invoice Date': [invoice_date],
              'Trade Name': [trade_name],
              'Period': [period],
              'Convenience Fee': [convenience_fee],
              'Commission': [commission],
              'Taxable Amount': [taxable_amount],
              'SGST @ 9.00%': [sgst],
              'CGST @ 9.00%': [cgst],
              'Total Amount': [total_amount]}
        # Convert the dictionary to a pandas DataFrame
      df = pd.DataFrame(data, index=[0])

      # Add the 'Source File' column to the DataFrame
      df.insert(0, 'Source File', os.path.basename(file), allow_duplicates=False)

      # Append the DataFrame to the data DataFrame
      zom_data = pd.concat([zom_data, df], ignore_index=True)



#@title adding other colms -2
zom_data['Trade Name'].replace(to_replace=" State Code : 29",value="",regex=True,inplace=True)
zom_data['Period'].replace(to_replace="To",value="-",regex=True,inplace=True)
zom_data['Payout Period Month']=zom_data['Period'].str.split('-',expand=True)[1]
f = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun', '07': 'July', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}

zom_data['Invoice Month']=zom_data['Invoice Date'].str.split('-',expand=True)[1].map(f)
#zom_data.rename(columns={'Trade Name':'Res_Name'},inplace=True)
# zom_data['Res_Name']=zom_data['Trade Name']
g={value:key for key,value in f.items()}

zom_data['Invoice_Month_Serial']=zom_data['Invoice Month'].map(g)
# zom_data['Invoice_Number']=zom_data['Invoice No']
print('Total Files - ',count)
zom_data.head(2)

Total Files -  4


,Source File,Invoice No,Invoice Date,Trade Name,Period,Convenience Fee,Commission,Taxable Amount,SGST @ 9.00%,CGST @ 9.00%,Total Amount,Payout Period Month,Invoice Month,Invoice_Month_Serial
0,Z24-KAOO-018397_18562485_Apr2023.pdf,Z24-KAOO-018397,2023-04-30,Chinese Chaska,01-Apr-2023 - 30-Apr-2023,"2,408.47",27425.72,29834.19,"2,685.07","2,685.07",35204.34,Apr,Apr,04
1,Z24-KAOO-017181_20017442_Apr2023.pdf,Z24-KAOO-017181,2023-04-30,Mann Food Court,01-Apr-2023 - 30-Apr-2023,"4,192.94",47747.30,51940.24,"4,674.62","4,674.62",61289.48,Apr,Apr,04


In [ ]:
#@title Pushing the data to Sheet

wc=gc.open_by_key('1zl-03wmWWDqF0yNN9w223KEQWiy877FJJ23EcjUCvNA').worksheet('Zomato_Invoices')

existing=gd.get_as_dataframe(wc)
update=existing.append(zom_data)

gd.set_with_dataframe(wc,update)


<ipython-input-4-3b2bc761b920>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  update=existing.append(zom_data)


In [ ]:
#@title Move invoices files to their respective,year month and rest_name folders

# replace with the path to your folder of restaurant files
source_path = '/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Zomato/Zomato Invoices' 

dest_path = '/content/drive/MyDrive/02_Aggregator_Invoices/01_Zomato_Invoices'


month_serial = {'01': 'Jan',
                '02': 'Feb',
                '03': 'Mar',
                '04': 'Apr',
                '05': 'May',
                '06': 'Jun',
                '07': 'Jul',
                '08': 'AUg',
                '09': 'Sep',
                '10': 'Oct',
                '11': 'Nov',
                '12':'Dec'}

restaurant_id_names = {
    "19461401": "Alpha Biryani",
    "19462922": "Biryani Box",
    "18387409": "Dilli Darbar",
    "19160468": "Roj Ka Khana",
    "18562485": "Chinese Chaska",
    "19524149": "Valluvan Cafe",
    "20017442": "Mann Food Court (NEW)",
    "19624944": "URUVALU BIRYANI",
    "20321016": "MFC Juice & Cafe",
    "20340609": "Koolerz",
    "20392581": "Grill Factory",
    "20395465": "Dilli Chaat"
}



# iterate over each file in the folder
for filename in os.listdir(source_path):
    if filename.startswith('Zomato_Ad_Invoices'):
      continue
    if filename.endswith(".pdf"):

      file_parts = filename.split("_")
      restaurant_id = filename.split("_")[1]
      month_name= filename.split("_")[2][:3]
      year= filename.split("_")[2][3:7]

      restaurant_name = restaurant_id_names.get(str(restaurant_id), "Unknown")

      # create the necessary directories if they don't already exist
      year_folder_path = os.path.join(dest_path, year)
      if not os.path.exists(year_folder_path):
          os.mkdir(year_folder_path)

      month_folder_path = os.path.join(year_folder_path, month_name)
      if not os.path.exists(month_folder_path):
          os.mkdir(month_folder_path)

      restaurant_folder_path = os.path.join(month_folder_path, restaurant_name)
      if not os.path.exists(restaurant_folder_path):
          os.mkdir(restaurant_folder_path)

      # move the file to the appropriate directory
      src_file_path = os.path.join(source_path, filename)
      dest_file_path = os.path.join(restaurant_folder_path, filename)
      shutil.move(src_file_path, dest_file_path)


# Swiggy Invoices

In [ ]:
#@title If files not in folder then move files from one folder to another 
des = '/content/drive/MyDrive/02_Aggregator_Invoices/02_Swiggy_Invoices'
sor = '/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Invoices reports/Swiggy Invoices'

count = 0

# Loop through each file in the source directory
for file in os.listdir(sor):
    src_path = os.path.join(sor, file)
    dest_path = os.path.join(des, file)
    if not os.path.exists(dest_path): 
#        shutil.move(src_path, dest_path)
        count += 1

# Print the total number of files moved
print(f"{count} files moved")

7 files moved


In [ ]:
#@title Swiggy Renaming pdf files to Standard Name



path2='/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Swiggy/Swiggy Invoices' # paste ur path link



# ******************************************************************************************* MAIN CODE **********************************************************************************************************

f = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun', '07': 'July', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}
files = glob.glob(os.path.join(path2, "*.pdf"))

count = 0

data2=[]

for file in files:
    with open(file, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)
        text = ""
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
        
            # Extract required information using regular expressions
            service_period_pattern = r'Service Period\s*:\s*([\d/-]+)\s*to\s*([\d/-]+)'
            service_period_match = re.search(service_period_pattern, text)
            service_period_start = service_period_match.group(1)
            service_period_end = service_period_match.group(2)

            restaurant_id_pattern = r'Restaurant \/ Store ID\s*:\s*(\d+)'
            restaurant_id_match = re.search(restaurant_id_pattern, text)
            restaurant_id = restaurant_id_match.group(1)

        data2.append({'Restaurant ID': restaurant_id,
                      'Start Date': service_period_start,
                      'End Date': service_period_end})
        
        for d in data2:
          month, year = d['End Date'].split('/')[-2:]
          month_name = f.get(month, 'Unknown')
          d['Month-Year'] = f"{month_name}{year}"

        for d in data2:
          start_day = d['Start Date'].split('/')[0]
          d['Start_day'] = f"{start_day}"

        for d in data2:
          end_day = d['End Date'].split('/')[0]
          d['End_day'] = f"{end_day}"

        for i in data2:
            oldname = os.path.join(path2, file)        
            split_name = os.path.splitext(os.path.basename(file))[0][:16]
            new_name = split_name + "_" + str(i['Restaurant ID'])+"_"+str(i['Start_day'])+str(i['Month-Year'])+"-"+str(i['End_day'])+str(i['Month-Year']) + '.pdf'
            newname = os.path.join(path2, new_name)     
        os.rename(oldname, newname)
        count+=1
print(count)
# (: worked

13


In [ ]:
#@title Main Code DC Swiggy invoices - 1


path2='/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Swiggy/Swiggy Invoices'






# ******************************************************************************************* MAIN CODE *******************************************************************

files = glob.glob(os.path.join(path2, "*.pdf"))

count = 0

swig_data=pd.DataFrame()

for file in files:
  with open(file, 'rb') as pdf_file:
      pdf_reader = PdfReader(pdf_file)
      num_pages = len(pdf_reader.pages)
      text = ""
      count+=1
      for page_num in range(num_pages):
          page = pdf_reader.pages[page_num]
          text += page.extract_text()

      # Extract the fields using string manipulation functions
      invoice_date = text.split('Invoice Date :')[1].split('\n')[0].strip()
      invoice_type = text.split('Invoice Type :')[1].split('\n')[0].strip()
      invoice_number = text.split('Invoice Number :')[1].split('\n')[0].strip()
      restaurant_name = text.split('Restaurant / Store Name :')[1].split('\n')[0].strip()
      subtotal = text.split('SubTotal')[1].split('\n')[0].strip()
      #other_charges = text.split('of Discount')[1].split('\n')[0].strip()
      grand_total = text.split('Grand Total')[1].split('\n')[0].strip()

      service_period_pattern = r'Service Period\s*:\s*([\d/-]+)\s*to\s*([\d/-]+)'
      service_period_match = re.search(service_period_pattern, text)
      service_period_start = service_period_match.group(1)
      service_period_end = service_period_match.group(2)

      discount_value1 = re.search(r'Discount([\d,\.]+)', text)
      #other_charges = discount_value.group(1)
      if discount_value1:
        other_charges = discount_value1.group(1)
      else:
        other_charges=0

      # Create a dictionary of the extracted fields
      data = {"service period start":service_period_start,
              "Service period end":service_period_end,
              'Invoice Date': invoice_date,
              'Invoice Type': invoice_type,
              'Invoice Number': invoice_number,
              'Restaurant / Store Name': restaurant_name,
              'SubTotal': subtotal,
              'Other Charges - Reimbursement of Discount': other_charges,
              'Grand Total': grand_total}

      # Convert the dictionary to a pandas DataFrame
      df = pd.DataFrame(data, index=[0])

      # Add the 'Source File' column to the DataFrame
      df.insert(0, 'Source File', os.path.basename(file), allow_duplicates=False)

      # Append the DataFrame to the data DataFrame
      swig_data = pd.concat([swig_data, df], ignore_index=True)


#@title Adding Other colms -2
swig_data[['Base Amount','Assessable value','CGST Amount','SGST Amount','IGST Amount','Comp CESS Amount','State CESS Amount','Total Amount (Rs.)']]=swig_data['SubTotal'].str.split(' ',expand=True)

swig_data=swig_data[['Source File', 'service period start', 'Service period end','Invoice Date', 'Invoice Type', 'Invoice Number','Restaurant / Store Name','Base Amount', 'Assessable value', 'CGST Amount', 'SGST Amount',
       'IGST Amount', 'Comp CESS Amount', 'State CESS Amount','Total Amount (Rs.)','Other Charges - Reimbursement of Discount', 'Grand Total']]

f = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun', '07': 'July', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}
swig_data['Payout Period Month']=swig_data['Service period end'].str.split("/",expand=True)[1].map(f)
swig_data['Invoice Month']=swig_data['Invoice Date'].str.split("-",expand=True)[1].map(f)
swig_data['Invoice Date']=swig_data['Invoice Date'].str.slice(stop=10)


g={value:key for key,value in f.items()}
swig_data['Payout_Month_Serial']=swig_data['Payout Period Month'].map(g)

swig_data['Invoice_Month_Serial']=swig_data['Invoice Month'].map(g)

swig_data['Payout_Period']=swig_data['service period start'].str.slice(stop=2)+" "+swig_data['Payout Period Month']+" "+swig_data['service period start'].str.slice(start=-4)+"-"+swig_data['Service period end'].str.slice(stop=2)+" "+swig_data['Payout Period Month']+" "+swig_data['Service period end'].str.slice(start=-4)

print('Total Files - ',count)
swig_data.head(2)

Total Files -  13


,Source File,service period start,Service period end,Invoice Date,Invoice Type,Invoice Number,Restaurant / Store Name,Base Amount,Assessable value,CGST Amount,...,Comp CESS Amount,State CESS Amount,Total Amount (Rs.),Other Charges - Reimbursement of Discount,Grand Total,Payout Period Month,Invoice Month,Payout_Month_Serial,Invoice_Month_Serial,Payout_Period
0,taco Tax_Invoice_600278_23Apr2023-30Apr2023.pdf,23/04/2023,30/04/2023,2023-05-05,INV,230505FS29000115,Dilli Chaat,"7,798.79","7,798.79",701.891,...,0,0,"9,202.572","7,029.69","16,232.262",Apr,May,04,05,23 Apr 2023-30 Apr 2023
1,taco Tax_Invoice_444423_23Apr2023-30Apr2023.pdf,23/04/2023,30/04/2023,2023-05-04,INV,230504FS29009043,URUVALU BIRYANI,"15,643.11","15,643.11","1,407.88",...,0,0,"18,458.87","22,326.33","40,785.2",Apr,May,04,05,23 Apr 2023-30 Apr 2023


In [ ]:
#@title Main Code DC Swiggy invoices - line wise with discriptions


path2='/content/drive/MyDrive/02_Aggregator_Invoices/02_Swiggy_Invoices/2023/Apr'


files = glob.glob(os.path.join(path2, "*.pdf"))

count = 0
data = {}
swig_data=pd.DataFrame()

for root, dir, files in os.walk(path2):
    for file in files:
        with open(os.path.join(root, file), 'rb') as pdf_file:
          pdf_reader = PdfReader(pdf_file)
          num_pages = len(pdf_reader.pages)
          text = ""
          count+=1
          for page_num in range(num_pages):
              page = pdf_reader.pages[page_num]
              text += page.extract_text()

          # Extract the fields using string manipulation functions
          invoice_date = text.split('Invoice Date :')[1].split('\n')[0].strip()
          invoice_type = text.split('Invoice Type :')[1].split('\n')[0].strip()
          invoice_number = text.split('Invoice Number :')[1].split('\n')[0].strip()
          restaurant_name = text.split('Restaurant / Store Name :')[1].split('\n')[0].strip()
          subtotal = text.split('SubTotal')[1].split('\n')[0].strip()
          #other_charges = text.split('of Discount')[1].split('\n')[0].strip()
          grand_total = text.split('Grand Total')[1].split('\n')[0].strip()

          service_period_pattern = r'Service Period\s*:\s*([\d/-]+)\s*to\s*([\d/-]+)'
          service_period_match = re.search(service_period_pattern, text)
          service_period_start = service_period_match.group(1)
          service_period_end = service_period_match.group(2)

          discount_value1 = re.search(r'Discount([\d,\.]+)', text)
          #other_charges = discount_value.group(1)
          if discount_value1:
            other_charges = discount_value1.group(1)
          else:
            other_charges=0

          # Extract the text between "AmountTotal Amount (Rs.)" and "SubTotal"
          descriptions_match = re.search(r'AmountTotal Amount \(Rs\.\)\n(.+?)\nSubTotal', text, re.DOTALL)
          if descriptions_match:
              descriptions_text = descriptions_match.group(1)
              descriptions = re.findall(r'\d+[A-Za-z\s]+', descriptions_text)
          else:
              descriptions = []

          # Create a dictionary of the extracted fields
          data = {
              "service period start": service_period_start,
              "Service period end": service_period_end,
              'Invoice Date': invoice_date,
              'Invoice Type': invoice_type,
              'Invoice Number': invoice_number,
              'Restaurant / Store Name': restaurant_name,
              'SubTotal': subtotal,
              'Other Charges - Reimbursement of Discount': other_charges,
              'Grand Total': grand_total,
              'Descriptions': descriptions
          }

          # Convert the dictionary to a pandas DataFrame
          df = pd.DataFrame(data)

          # Add the 'Source File' column to the DataFrame
          df.insert(0, 'Source File', os.path.basename(file), allow_duplicates=False)

          # Append the DataFrame to the data DataFrame
          swig_data = pd.concat([swig_data, df], ignore_index=True)


new_columns = ['Base Amount', 'Assessable value', 'CGST Amount', 'SGST Amount',
               'IGST Amount', 'Comp CESS Amount', 'State CESS Amount', 'Total Amount (Rs.)']
swig_data[new_columns] = swig_data['SubTotal'].str.split(' ', expand=True)
swig_data.drop('SubTotal', axis=1, inplace=True)


swig_data = swig_data[swig_data['Descriptions'].str.len() >= 5]
swig_data['Descriptions'] = swig_data['Descriptions'].str.replace(r'^\d+', '').str.strip().str.replace('\n', '')


f = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun', '07': 'July', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}
swig_data['Payout Period Month']=swig_data['Service period end'].str.split("/",expand=True)[1].map(f)
swig_data['Invoice Month']=swig_data['Invoice Date'].str.split("-",expand=True)[1].map(f)
swig_data['Invoice Date']=swig_data['Invoice Date'].str.slice(stop=10)


g={value:key for key,value in f.items()}
swig_data['Payout_Month_Serial']=swig_data['Payout Period Month'].map(g)

swig_data['Invoice_Month_Serial']=swig_data['Invoice Month'].map(g)

swig_data['Payout_Period']=swig_data['service period start'].str.slice(stop=2)+" "+swig_data['Payout Period Month']+" "+swig_data['service period start'].str.slice(start=-4)+"-"+swig_data['Service period end'].str.slice(stop=2)+" "+swig_data['Payout Period Month']+" "+swig_data['Service period end'].str.slice(start=-4)

print('Total Files - ',count)

swig_data.head(2)

Total Files -  51


<ipython-input-20-c3457de30160>:84: FutureWarning: The default value of regex will change from True to False in a future version.
  swig_data['Descriptions'] = swig_data['Descriptions'].str.replace(r'^\d+', '').str.strip().str.replace('\n', '')


,Source File,service period start,Service period end,Invoice Date,Invoice Type,Invoice Number,Restaurant / Store Name,Other Charges - Reimbursement of Discount,Grand Total,Descriptions,...,SGST Amount,IGST Amount,Comp CESS Amount,State CESS Amount,Total Amount (Rs.),Payout Period Month,Invoice Month,Payout_Month_Serial,Invoice_Month_Serial,Payout_Period
0,taco Tax_Invoice_444423_16Apr2023-22Apr2023.pdf,16/04/2023,22/04/2023,2023-04-28,INV,230428FS29000134,URUVALU BIRYANI,"13,399.59","22,598.516",Service Fee,...,701.613,0,0,0,"9,198.926",Apr,Apr,04,04,16 Apr 2023-22 Apr 2023
1,taco Tax_Invoice_444423_16Apr2023-22Apr2023.pdf,16/04/2023,22/04/2023,2023-04-28,INV,230428FS29000134,URUVALU BIRYANI,"13,399.59","22,598.516",OTH,...,701.613,0,0,0,"9,198.926",Apr,Apr,04,04,16 Apr 2023-22 Apr 2023


In [ ]:
#@title Main Code DC Swiggy invoices - line wise with discriptions


path2='/content/drive/MyDrive/02_Aggregator_Invoices/02_Swiggy_Invoices/2023'


files = glob.glob(os.path.join(path2, "*.pdf"))

count = 0
data = {}
swig_data=pd.DataFrame()

for root, dir, files in os.walk(path2):
    for file in files:
        with open(os.path.join(root, file), 'rb') as pdf_file:
          pdf_reader = PdfReader(pdf_file)
          num_pages = len(pdf_reader.pages)
          text = ""
          count+=1
          for page_num in range(num_pages):
              page = pdf_reader.pages[page_num]
              text += page.extract_text()

          # Extract the fields using string manipulation functions
          invoice_date = text.split('Invoice Date :')[1].split('\n')[0].strip()
          invoice_type = text.split('Invoice Type :')[1].split('\n')[0].strip()
          invoice_number = text.split('Invoice Number :')[1].split('\n')[0].strip()
          restaurant_name = text.split('Restaurant / Store Name :')[1].split('\n')[0].strip()
          subtotal = text.split('SubTotal')[1].split('\n')[0].strip()
          #other_charges = text.split('of Discount')[1].split('\n')[0].strip()
          grand_total = text.split('Grand Total')[1].split('\n')[0].strip()

          service_period_pattern = r'Service Period\s*:\s*([\d/-]+)\s*to\s*([\d/-]+)'
          service_period_match = re.search(service_period_pattern, text)
          service_period_start = service_period_match.group(1)
          service_period_end = service_period_match.group(2)

          discount_value1 = re.search(r'Discount([\d,\.]+)', text)
          #other_charges = discount_value.group(1)
          if discount_value1:
            other_charges = discount_value1.group(1)
          else:
            other_charges=0

          # Extract the text between "AmountTotal Amount (Rs.)" and "SubTotal"
          descriptions_match = re.search(r'AmountTotal Amount \(Rs\.\)\n(.+?)\nSubTotal', text, re.DOTALL)
          if descriptions_match:
              descriptions_text = descriptions_match.group(1)
              # descriptions = re.findall(r'\d+[A-Za-z\s]+', descriptions_text)

          # Create a dictionary of the extracted fields
          data = {
              "service period start": service_period_start,
              "Service period end": service_period_end,
              'Invoice Date': invoice_date,
              'Invoice Type': invoice_type,
              'Invoice Number': invoice_number,
              'Restaurant / Store Name': restaurant_name,
              'SubTotal': subtotal,
              'Other Charges - Reimbursement of Discount': other_charges,
              'Grand Total': grand_total,
              'Descriptions': descriptions_text
          }

          # Convert the dictionary to a pandas DataFrame
          df = pd.DataFrame(data,index=[0])

          # Add the 'Source File' column to the DataFrame
          df.insert(0, 'Source File', os.path.basename(file), allow_duplicates=False)

          # Append the DataFrame to the data DataFrame
          swig_data = pd.concat([swig_data, df], ignore_index=True)


swig_data.head(2)

,Source File,service period start,Service period end,Invoice Date,Invoice Type,Invoice Number,Restaurant / Store Name,SubTotal,Other Charges - Reimbursement of Discount,Grand Total,Descriptions
0,taco Tax_Invoice_313834_19Feb2023-25Feb2023.pdf,19/02/2023,25/02/2023,2023-03-01 19:01:11,INV,230301FS29007550,Biryani Hub,"924.86 924.86 83.237 83.237 0 0 0 1,091.335",969.05,"2,060.385",1Service Fee 996211 OTH 1 924.86 924.86 0 924....
1,taco Tax_Invoice_313834_26Feb2023-28Feb2023.pdf,26/02/2023,28/02/2023,2023-03-04 02:52:00,INV,230304FS29000596,Biryani Hub,627.8 627.8 56.502 56.502 0 0 0 740.804,387.2,"1,128.004",1Service Fee 996211 OTH 1 262.8 262.8 0 262.8 ...


In [ ]:
#@title Pushing the data to sheet


swiggy_sheetID_where_data_need_tobe_saved= '1zl-03wmWWDqF0yNN9w223KEQWiy877FJJ23EcjUCvNA'




# ******************************************************************** MAIN CODE ************************************************************************

wc=gc.open_by_key(f'{swiggy_sheetID_where_data_need_tobe_saved}').worksheet('Swiggy_Invoices')

existing=gd.get_as_dataframe(wc)
update=existing.append(swig_data)
gd.set_with_dataframe(wc,update)


<ipython-input-4-b18d973e2c32>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  update=existing.append(swig_data)


In [2]:
#@title Move invoices files to their respective month and rest_name folders 

# replace with the path to your folder of restaurant files
source_path = '/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Swiggy/Swiggy Invoices' 

dest_path="/content/drive/MyDrive/02_Aggregator_Invoices/02_Swiggy_Invoices"



# ************************************************************************** MAIN CODE ********************************************************************

restaurant_id_names = {
    "313834": "Biryani HUB",
    "310274": "Biryani Box",
    "4780": "Dilli Darbar",
    "180796": "Biryani Box",
    "408": "Alpha Biryani",
    "180801": "Roj Ka Khana",
    "231392": "Chinese Chaska",
    "330114": "Valluvan Cafe",
    "445384": "Mann Food Court (NEW)",
    "444423": "URUVALU BIRYANI",
    "570269": "MFC Juice & Cafe",
    "570268": "Koolerz",
    "593472": "Grill Factory",
    "600278": "Dilli Chaat"}


# iterate over each file in the folder
for filename in os.listdir(source_path):
    if filename.endswith(".pdf"):

      file_parts = filename.split("_")
      restaurant_id = [part for part in file_parts if part.isdigit()][0]

      restaurant_name = restaurant_id_names.get(str(restaurant_id), "Unknown")


      month_name= filename.split("-")[1][2:5]
      year= filename.split("_")[3][5:9]


      year_folder_path = os.path.join(dest_path, year)
      if not os.path.exists(year_folder_path):
          os.mkdir(year_folder_path)

      month_folder_path = os.path.join(year_folder_path, month_name)
      if not os.path.exists(month_folder_path):
          os.mkdir(month_folder_path)

      restaurant_source_path = os.path.join(month_folder_path, restaurant_name)
      if not os.path.exists(restaurant_source_path):
          os.mkdir(restaurant_source_path)

      # move the file to the appropriate directory
      src_file_path = os.path.join(source_path, filename)
      dest_file_path = os.path.join(restaurant_source_path, filename)
      shutil.move(src_file_path, dest_file_path)

print('Done')


Done


#Thrive

In [ ]:
#@title if files not in folder then move files from one folder to another 
destination = '/content/drive/MyDrive/02_Aggregator_Invoices/03_Thrive_Invoices'             #destination path
source = '/content/drive/MyDrive/Partners Reports Temporary Save 🧩/Thrive/Thrive reports/Thrive Invoices'    #source path



# ******************************************************** MAin CODE **************************************************************
count = 0

# Loop through each file in the source directory
for file in os.listdir(source):
    src_path = os.path.join(source, file)
    dest_path = os.path.join(destination, file)
    if not os.path.exists(dest_path): 
        shutil.move(src_path, dest_path)
        count += 1

# Print the total number of files moved
print(f"{count} files moved")

4 files moved


In [ ]:
#@title Changing names of report files to Standard names



folder='/content/drive/MyDrive/02_Aggregator_Invoices/03_Thrive_Invoices'





# ******************************************************************************************* MAIN CODE **********************************************************************************************************

count=0
files = glob.glob(os.path.join(folder , "*.pdf"))

for file in files:
  with open(file, 'rb') as pdf_file:
      pdf_reader = PdfReader(pdf_file)
      num_pages = len(pdf_reader.pages)
      text = ""
      for page_num in range(num_pages):
          page = pdf_reader.pages[page_num]
          text += page.extract_text()
          invoice_num_pattern = r"#\s*([A-Z0-9\-]+)"
          match = re.search(invoice_num_pattern, text)
          if match:
              invoice_num = match.group(1)

          # Extract restaurant name
          restaurant_pattern = r"Bill\s+To\s+([\w\s.,#-]+?)\n"
          match = re.search(restaurant_pattern, text)
          if match:
              restaurant_name = match.group(1).strip().split("Restaurant name: ")[-1]

          date_pattern = r"Invoice\s+Date\s*:\s*(\d{2}/\d{2}/\d{4})"
          match = re.search(date_pattern, text)
          if match:
            invoice_date = match.group(1)
          inv=invoice_date.replace("/","-")


          oldname = os.path.join(folder, file)        
          new_name = invoice_num+"_"+restaurant_name+"_"+ inv+ '.pdf'
          newname = os.path.join(folder, new_name)  

      os.rename(oldname, newname)
      count+=1
print(count)


18


In [ ]:
#@title Main Code DC Thrive invoices - 1


path2='/content/drive/MyDrive/02_Aggregator_Invoices/03_Thrive_Invoices/2023/Apr' # paste ur path link







# **************************************************************************************************** MAIN CODE ******************************************************************************************************************

files = glob.glob(os.path.join(path2, "*.pdf"))

count = 0

thrive_data=pd.DataFrame()

for root,dirs, folders in os.walk(path2):
  for folder in folders:
    file_path=os.path.join(root,folder)
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        num_pages = len(pdf_reader.pages)
        text = ""
        count+=1
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

        # Extract the required information using regular expressions
        bill_to = re.search(r'Bill To\s+([\w\s.,#]+)\s+([\w\s.,#]+)\s+([\w\s.,#]+)', text)
        invoice_date = re.search(r'Invoice Date\s+:\s+(\d{2}/\d{2}/\d{4})', text)
        invoice_number = re.search(r'# INV-(\d+)', text)
        gstin = re.search(r'GSTIN\s+(\w+)', text)
        due_date = re.search(r'Due Date\s+:\s+(\d{2}/\d{2}/\d{4})', text)
        # transaction_fee = re.search(r'1\s+Transaction Fee\n([\w\s,.#\n]+)\n', text)
        # delivery_fee = re.search(r'2\s+Delivery Fee\n([\w\s,.#\n]+)\n', text)
        # razorpay_fee = re.search(r'3\s+Razorpay Fee\n([\w\s,.#\n]+)\n', text)
        igst = re.search(r'IGST18 \(18%\).*?(\d+\.\d+)', text, re.DOTALL)
        total = re.search(r'Total Rs\.([0-9,.]+)', text)
        # balance_due = re.search(r'Balance Due Rs\.([0-9,.]+)', text)

        # Create a dictionary of the extracted information
        data = {'Bill To': bill_to.group(1),
                'City': bill_to.group(2),
                'State': bill_to.group(3),
                'Invoice Date': invoice_date.group(1),
                'Invoice Number': invoice_number.group(1),
                'GSTIN': gstin.group(1),
                'Due Date': due_date.group(1),
                'IGST18 (18%)': igst.group(1) if igst else None,
                'Total': total.group(1)}

        # Convert the dictionary to a pandas DataFrame
        df = pd.DataFrame(data, index=[0])

        # Add the 'Source File' column to the DataFrame
        df.insert(0, 'Source File', os.path.basename(file), allow_duplicates=False)

        # Append the DataFrame to the data DataFrame
        thrive_data = pd.concat([thrive_data, df], ignore_index=True)
      
print('Total Files - ',count)




f = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun', '07': 'July', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}
thrive_data['Invoice Month']=thrive_data['Invoice Date'].str.split("/",expand=True)[1].map(f)


thrive_data['Res_Name']=thrive_data['Bill To'].str.split('\n',expand=True)[0]

thrive_data['Invoice Number']='INV-'+thrive_data['Invoice Number']
thrive_data.drop(columns=['City','State','Bill To'],inplace=True)

thrive_data['GSTIN']=thrive_data['GSTIN'].replace(to_replace='Invoice', value='', inplace=False, regex=True)
thrive_data['Invoice_Number']=thrive_data['Invoice Number']

g={value:key for key,value in f.items()}

thrive_data['Invoice_Month_Serial']=thrive_data['Invoice Month'].map(g)

thrive_data.head(2)

Total Files -  4


,Source File,Invoice Date,Invoice Number,GSTIN,Due Date,IGST18 (18%),Total,Invoice Month,Res_Name,Invoice_Number,Invoice_Month_Serial
0,taco Tax_Invoice_444423_05042023_230405FS29006...,30/04/2023,INV-033187,27AADCH9016M1Z9,15/05/2023,776.77,"5,092.20",Apr,Alpha Biryani,INV-033187,04
1,taco Tax_Invoice_444423_05042023_230405FS29006...,30/04/2023,INV-033183,27AADCH9016M1Z9,15/05/2023,43.02,282.02,Apr,Chinese Chaska,INV-033183,04


In [ ]:
#@title Pushing the data to sheets



thrive_sheeID_where_data_need_to_be_saved='1zl-03wmWWDqF0yNN9w223KEQWiy877FJJ23EcjUCvNA'



# ******************************************************************** MAIN CODE ************************************************************************

wc=gc.open_by_key(f'{thrive_sheeID_where_data_need_to_be_saved}').worksheet('Thrive_Invoices')


existing=gd.get_as_dataframe(wc)
update=existing.append(thrive_data)
gd.set_with_dataframe(wc,update)


<ipython-input-13-e9dedc06a2fa>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  update=existing.append(thrive_data)


In [ ]:
#@title Move invoices files to their respective,year month and rest_name folders

# replace with the path to your folder of restaurant files
source_path = "/content/drive/MyDrive/01_Sales_Reports/D_Thrive/Frm_April_Summary_report_files"

dest_path="/content/drive/MyDrive/02_Aggregator_Invoices/02_Swiggy_Invoices"



# ************************************************************************** MAIN CODE ********************************************************************

month_serial = {'01': 'Jan',
                '02': 'Feb',
                '03': 'Mar',
                '04': 'Apr',
                '05': 'May',
                '06': 'Jun',
                '07': 'Jul',
                '08': 'AUg',
                '09': 'Sep',
                '10': 'Oct',
                '11': 'Nov',
                '12':'Dec'}


# iterate over each file in the folder
for filename in os.listdir(source_path):
    if filename.endswith(".pdf"):

      file_parts = filename.split("_")

      month_serail = filename.split("_")[2][3:5]
      month_name= month_serial.get(month_serail, "Unknown")
      year= filename.split("_")[2][6:10]

      restaurant_name = filename.split("_")[1]

      # create the necessary directories if they don't already exist
      year_folder_path = os.path.join(dest_path, year)
      if not os.path.exists(year_folder_path):
          os.mkdir(year_folder_path)

      month_folder_path = os.path.join(year_folder_path, month_name)
      if not os.path.exists(month_folder_path):
          os.mkdir(month_folder_path)

      restaurant_folder_path = os.path.join(month_folder_path, restaurant_name)
      if not os.path.exists(restaurant_folder_path):
          os.mkdir(restaurant_folder_path)

      # move the file to the appropriate directory
      src_file_path = os.path.join(folder_path, filename)
      dest_file_path = os.path.join(restaurant_folder_path, filename)
      shutil.move(src_file_path, dest_file_path)

